In [1]:
import numpy as np
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split , KFold
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
import pandas as pd
import math
from bayes_opt import BayesianOptimization
import random
import sklearn
from sklearn.metrics import mean_squared_error
import tensorflow as tf

In [2]:

# Step 1: Load your dataset. Make sure you have a CSV file with relevant features and effort (target) column.
# Replace 'your_dataset.csv' with your actual dataset file name and adjust the column names accordingly.
desharnais=pd.read_csv("C:\\Users\\Asus\\Desktop\\Tehran university\\Seminar\\Datasets\\desharnais_dataset.csv",header=None,index_col=0)
columns_desharnais=['TeamExp','ManagerExp','YearEnd','Length','Effort','Transactions','Entities','PointsAdjust','Envergure','PointsNonAjust','Language']
desharnais.set_axis(columns_desharnais,axis='columns',inplace=True)
desharnais.rename_axis("Features", axis=1,inplace=True)
desharnais.rename_axis("Projects", axis=0,inplace=True)
dataset = desharnais

# Step 2: Prepare the data.
X = dataset.drop(columns=['Effort']).values  # Features
y = dataset['Effort'].values  # Target (effort)

seed_value = 42
random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)

# Step 2: Preprocess Data
#scaler = StandardScaler()
#X_scaled = scaler.fit_transform(X)

# Step 3: Feature Selection using Harmony Search
def objective_function(subset):
    selected_features = [feature for feature, is_selected in zip(range(X.shape[1]), subset) if is_selected]
    if len(selected_features) == 0:
        return float('-inf')  # Penalize subsets with no selected features
    
    num_folds = 5
    kf = KFold(n_splits=num_folds, shuffle=True, random_state=seed_value)
    
    X_selected = X[:, selected_features]
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_selected)
    error_list=[]
    
    for train_index, val_index in kf.split(X_scaled):
        X_train, X_test = X_scaled[train_index], X_scaled[val_index]
        y_train, y_test = y[train_index], y[val_index]
    
        # Step 3: Build the ANN model.
        model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(units=32, activation='relu', input_shape=(X_train.shape[1],)),
        tf.keras.layers.Dense(units=16, activation='relu'),
        tf.keras.layers.Dense(units=1)  # Output layer with a single unit for regression.
    ])

        # Step 4: Compile the model.
        model.compile(optimizer='adam', loss='mean_squared_error')

        # Step 5: Train the model.
        model.fit(X_train, y_train, epochs=5, batch_size=8, verbose=1)

        # Step 6: Evaluate the model.
        y_pred = model.predict(X_test)
    
        error = np.mean(np.abs(y_pred - y_test))
        error_list.append(error)
    
    return 1 / (1 + np.mean(error_list))  # Fitness is the inverse of the error (higher is better)

#==================================================================================================================
# Harmony Search Parameters
num_features = X.shape[1]
hms = 20  # Harmony memory size
iterations = 100  # Number of iterations

# Initialize harmony memory
harmony_memory = np.random.randint(0, 2, size=(hms, num_features))

# Harmony Search Algorithm
for _ in range(iterations):
    rnd_choice=np.random.choice(hms)
    new_harmony = np.copy(harmony_memory[rnd_choice])
    for i in range(num_features):
        if np.random.rand() < 0.5:  # Adjust this probability based on your problem
            new_harmony[i] = 1 - new_harmony[i]
    current_obj = objective_function(harmony_memory[rnd_choice])
    new_obj = objective_function(new_harmony)
    if new_obj > current_obj:
        harmony_memory[rnd_choice] = new_harmony

# Get selected features based on final harmony memory
selected_features = harmony_memory[np.argmax([objective_function(i) for i in harmony_memory])]
print(selected_features)
# Step 4: Train SVR Model using selected features
#selected_X = X_scaled[:, np.array(selected_features) == 1]
#=============================================================================================================               
num_folds = 5
kf = KFold(n_splits=num_folds, shuffle=True, random_state=seed_value)
    
X_selected = X[:, selected_features]
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_selected)
mean_RMSE=[]
mean_MMRE=[]
mean_MAE=[]
    
for train_index, val_index in kf.split(X_scaled):
    X_train, X_test = X_scaled[train_index], X_scaled[val_index]
    y_train, y_test = y[train_index], y[val_index]
    
    # Step 3: Build the ANN model.
    model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(units=32, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(units=16, activation='relu'),
    tf.keras.layers.Dense(units=1)  # Output layer with a single unit for regression.
])

    # Step 4: Compile the model.
    model.compile(optimizer='adam', loss='mean_squared_error')

    # Step 5: Train the model.
    model.fit(X_train, y_train, epochs=5, batch_size=8, verbose=1)

    # Step 6: Evaluate the model.
    y_pred = model.predict(X_test)

    mae = mean_absolute_error(y_test, y_pred)
    mean_MAE.append(mae)


    # Calculate Mean Magnitude of Relative Error (MMRE)
    mmre = np.mean(np.abs((y_test - y_pred) / y_test))
    mean_MMRE.append(mmre)

    # Calculate the Root Mean Squared Error (RMSE) to assess the model's performance.
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mean_RMSE.append(rmse)
                
print(f"Mean Absolute Error mean: {np.mean(mean_MAE)}")               
print(f"Mean Magnitude of Relative Error mean (MMRE): {np.mean(mean_MMRE):.2f}")            
print(f"Root Mean Squared Error (RMSE) mean: {np.mean(mean_RMSE)}")
print(selected_features)


Epoch 1/5
8/8 [==============================] - 0s 1ms/step - loss: 47787920.0000
Epoch 2/5
8/8 [==============================] - 0s 1000us/step - loss: 47786236.0000
Epoch 3/5
8/8 [==============================] - 0s 1000us/step - loss: 47784652.0000
Epoch 4/5
8/8 [==============================] - 0s 857us/step - loss: 47783208.0000
Epoch 5/5
1/1 [==============================] - 0s 52ms/step
Epoch 1/5
9/9 [==============================] - 0s 875us/step - loss: 40639016.0000
Epoch 2/5
9/9 [==============================] - 0s 875us/step - loss: 40636872.0000
Epoch 3/5
9/9 [==============================] - 0s 875us/step - loss: 40634636.0000
Epoch 4/5
9/9 [==============================] - 0s 750us/step - loss: 40631896.0000
Epoch 5/5
1/1 [==============================] - 0s 32ms/step
Epoch 1/5
9/9 [==============================] - 0s 1ms/step - loss: 48812256.0000
Epoch 2/5
9/9 [==============================] - 0s 749us/step - loss: 48810140.0000
Epoch 3/5
9/9 [=============

9/9 [==============================] - 0s 1000us/step - loss: 41688452.0000
Epoch 3/5
9/9 [==============================] - 0s 876us/step - loss: 41685636.0000
Epoch 4/5
9/9 [==============================] - 0s 1ms/step - loss: 41682568.0000
Epoch 5/5
1/1 [==============================] - 0s 31ms/step
Epoch 1/5
8/8 [==============================] - 0s 1ms/step - loss: 47770704.0000
Epoch 2/5
8/8 [==============================] - 0s 999us/step - loss: 47768032.0000
Epoch 3/5
8/8 [==============================] - 0s 1000us/step - loss: 47765120.0000
Epoch 4/5
8/8 [==============================] - 0s 857us/step - loss: 47761916.0000
Epoch 5/5
1/1 [==============================] - 0s 32ms/step
Epoch 1/5
9/9 [==============================] - 0s 1ms/step - loss: 40643144.0000
Epoch 2/5
9/9 [==============================] - 0s 939us/step - loss: 40640944.0000
Epoch 3/5
9/9 [==============================] - 0s 875us/step - loss: 40638824.0000
Epoch 4/5
9/9 [=========================

9/9 [==============================] - 0s 875us/step - loss: 40638868.0000
Epoch 4/5
9/9 [==============================] - 0s 875us/step - loss: 40637344.0000
Epoch 5/5
1/1 [==============================] - 0s 33ms/step
Epoch 1/5
9/9 [==============================] - 0s 1ms/step - loss: 48809868.0000
Epoch 2/5
9/9 [==============================] - 0s 877us/step - loss: 48807832.0000
Epoch 3/5
9/9 [==============================] - 0s 875us/step - loss: 48805368.0000
Epoch 4/5
9/9 [==============================] - 0s 940us/step - loss: 48803032.0000
Epoch 5/5
1/1 [==============================] - 0s 32ms/step
Epoch 1/5
9/9 [==============================] - 0s 1ms/step - loss: 44858460.0000
Epoch 2/5
9/9 [==============================] - 0s 750us/step - loss: 44857076.0000
Epoch 3/5
9/9 [==============================] - 0s 874us/step - loss: 44855964.0000
Epoch 4/5
9/9 [==============================] - 0s 875us/step - loss: 44854860.0000
Epoch 5/5
1/1 [=========================

9/9 [==============================] - 0s 875us/step - loss: 44852180.0000
Epoch 5/5
1/1 [==============================] - 0s 33ms/step
Epoch 1/5
9/9 [==============================] - 0s 1ms/step - loss: 41699668.0000
Epoch 2/5
9/9 [==============================] - 0s 875us/step - loss: 41698372.0000
Epoch 3/5
9/9 [==============================] - 0s 1000us/step - loss: 41696932.0000
Epoch 4/5
9/9 [==============================] - 0s 1000us/step - loss: 41695352.0000
Epoch 5/5
1/1 [==============================] - 0s 32ms/step
Epoch 1/5
8/8 [==============================] - 0s 1ms/step - loss: 47781224.0000
Epoch 2/5
8/8 [==============================] - 0s 714us/step - loss: 47779164.0000
Epoch 3/5
8/8 [==============================] - 0s 1ms/step - loss: 47777004.0000
Epoch 4/5
8/8 [==============================] - 0s 714us/step - loss: 47774808.0000
Epoch 5/5
1/1 [==============================] - 0s 33ms/step
Epoch 1/5
9/9 [==============================] - 0s 1ms/step - 

1/1 [==============================] - 0s 32ms/step
Epoch 1/5
9/9 [==============================] - 0s 1ms/step - loss: 40639680.0000
Epoch 2/5
9/9 [==============================] - 0s 874us/step - loss: 40637796.0000
Epoch 3/5
9/9 [==============================] - 0s 875us/step - loss: 40635808.0000
Epoch 4/5
9/9 [==============================] - 0s 750us/step - loss: 40633644.0000
Epoch 5/5
1/1 [==============================] - 0s 31ms/step
Epoch 1/5
9/9 [==============================] - 0s 1ms/step - loss: 48813440.0000
Epoch 2/5
9/9 [==============================] - 0s 750us/step - loss: 48810864.0000
Epoch 3/5
9/9 [==============================] - 0s 875us/step - loss: 48807884.0000
Epoch 4/5
9/9 [==============================] - 0s 749us/step - loss: 48804936.0000
Epoch 5/5
1/1 [==============================] - 0s 34ms/step
Epoch 1/5
9/9 [==============================] - 0s 1ms/step - loss: 44853668.0000
Epoch 2/5
9/9 [==============================] - 0s 750us/step - 

...

9/9 [==============================] - 0s 768us/step - loss: 44857528.0000
Epoch 3/5
9/9 [==============================] - 0s 1000us/step - loss: 44855500.0000
Epoch 4/5
9/9 [==============================] - 0s 875us/step - loss: 44853412.0000
Epoch 5/5
1/1 [==============================] - 0s 36ms/step
Epoch 1/5
9/9 [==============================] - 0s 1ms/step - loss: 41694572.0000
Epoch 2/5
9/9 [==============================] - 0s 765us/step - loss: 41692020.0000
Epoch 3/5
9/9 [==============================] - 0s 875us/step - loss: 41689040.0000
Epoch 4/5
9/9 [==============================] - 0s 875us/step - loss: 41685616.0000
Epoch 5/5
1/1 [==============================] - 0s 35ms/step
Epoch 1/5
8/8 [==============================] - 0s 1000us/step - loss: 47786016.0000
Epoch 2/5
8/8 [==============================] - 0s 1000us/step - loss: 47784584.0000
Epoch 3/5
8/8 [==============================] - 0s 857us/step - loss: 47783384.0000
Epoch 4/5
8/8 [====================

8/8 [==============================] - 0s 856us/step - loss: 47789376.0000
Epoch 4/5
8/8 [==============================] - 0s 857us/step - loss: 47787724.0000
Epoch 5/5
1/1 [==============================] - 0s 35ms/step
Epoch 1/5
9/9 [==============================] - 0s 875us/step - loss: 40645312.0000
Epoch 2/5
9/9 [==============================] - 0s 875us/step - loss: 40643032.0000
Epoch 3/5
9/9 [==============================] - 0s 875us/step - loss: 40640868.0000
Epoch 4/5
9/9 [==============================] - 0s 875us/step - loss: 40638276.0000
Epoch 5/5
1/1 [==============================] - 0s 34ms/step
Epoch 1/5
9/9 [==============================] - 0s 1ms/step - loss: 48816084.0000
Epoch 2/5
9/9 [==============================] - 0s 875us/step - loss: 48813844.0000
Epoch 3/5
9/9 [==============================] - 0s 875us/step - loss: 48811084.0000
Epoch 4/5
9/9 [==============================] - 0s 750us/step - loss: 48808344.0000
Epoch 5/5
1/1 [=======================

9/9 [==============================] - 0s 750us/step - loss: 48810696.0000
Epoch 5/5
1/1 [==============================] - 0s 34ms/step
Epoch 1/5
9/9 [==============================] - 0s 1ms/step - loss: 44849924.0000
Epoch 2/5
9/9 [==============================] - 0s 750us/step - loss: 44847388.0000
Epoch 3/5
9/9 [==============================] - 0s 750us/step - loss: 44844468.0000
Epoch 4/5
9/9 [==============================] - 0s 625us/step - loss: 44841276.0000
Epoch 5/5
1/1 [==============================] - 0s 33ms/step
Epoch 1/5
9/9 [==============================] - 0s 876us/step - loss: 41701860.0000
Epoch 2/5
9/9 [==============================] - 0s 875us/step - loss: 41700396.0000
Epoch 3/5
9/9 [==============================] - 0s 875us/step - loss: 41698944.0000
Epoch 4/5
9/9 [==============================] - 0s 875us/step - loss: 41697348.0000
Epoch 5/5
1/1 [==============================] - 0s 34ms/step
Epoch 1/5
8/8 [==============================] - 0s 858us/ste

1/1 [==============================] - 0s 38ms/step
Epoch 1/5
8/8 [==============================] - 0s 857us/step - loss: 47788224.0000
Epoch 2/5
8/8 [==============================] - 0s 1ms/step - loss: 47785404.0000
Epoch 3/5
8/8 [==============================] - 0s 714us/step - loss: 47782700.0000
Epoch 4/5
8/8 [==============================] - 0s 1000us/step - loss: 47780196.0000
Epoch 5/5
1/1 [==============================] - 0s 37ms/step
Epoch 1/5
9/9 [==============================] - 0s 1ms/step - loss: 40642144.0000
Epoch 2/5
9/9 [==============================] - 0s 1ms/step - loss: 40640432.0000
Epoch 3/5
9/9 [==============================] - 0s 1ms/step - loss: 40638720.0000
Epoch 4/5
9/9 [==============================] - 0s 1ms/step - loss: 40636824.0000
Epoch 5/5
1/1 [==============================] - 0s 37ms/step
Epoch 1/5
9/9 [==============================] - 0s 1000us/step - loss: 48814864.0000
Epoch 2/5
9/9 [==============================] - 0s 875us/step - lo

9/9 [==============================] - 0s 1ms/step - loss: 48813252.0000
Epoch 2/5
9/9 [==============================] - 0s 1000us/step - loss: 48811216.0000
Epoch 3/5
9/9 [==============================] - 0s 1000us/step - loss: 48808808.0000
Epoch 4/5
9/9 [==============================] - 0s 875us/step - loss: 48806612.0000
Epoch 5/5
1/1 [==============================] - 0s 39ms/step
Epoch 1/5
9/9 [==============================] - 0s 1ms/step - loss: 44856292.0000
Epoch 2/5
9/9 [==============================] - 0s 1ms/step - loss: 44853988.0000
Epoch 3/5
9/9 [==============================] - 0s 1000us/step - loss: 44851396.0000
Epoch 4/5
9/9 [==============================] - 0s 1ms/step - loss: 44848812.0000
Epoch 5/5
1/1 [==============================] - 0s 39ms/step
Epoch 1/5
9/9 [==============================] - 0s 1000us/step - loss: 41700024.0000
Epoch 2/5
9/9 [==============================] - 0s 875us/step - loss: 41698456.0000
Epoch 3/5
9/9 [=========================